# Assignment 4: Data modeling and SQL for IMDB data
***

## Overview

A company named MOVIE GEN Corp. needs your help to create and access a movie database. You are interested in helping them create this database. In this assignment, you will load the IMDB data into a database and run queries on it to retrieve records. 

** OBJECTIVE **

In this assignment, you will first examine some flat data and develop an appropriate schema to efficiently represent that data in a SQL database.  You will then create that database, use Python to read and reformat the data and then use SQL commands to populate that database in an appropriate manner. Finally, you will query the database using different approaches and optimizations.

**TASKS**

1. Model data using star schema
3. Use SQL to create a database
4. Use Python to read data and create appropriate SQL commands to populate database 
4. Write relational algebra
5. Run SQL to retrieve records for your customers
7. Improve query performance
8. Database visualization

***
## Part 4.1: Data modeling

Build a **[star schema](https://en.wikipedia.org/wiki/Star_schema)** for the data contained in `all_data.csv`.

Let's first read the csv file and examine its contents.  The strings in `all_data.csv` use the `latin1` character encoding.  When you read in this file using `pandas`, make sure to include this encoding option.

In [ ]:
import pandas as pd

# read data in and show head
df = pd.read_csv('all_data.csv', encoding = 'latin1')
df.head()

There are 24 columns in the csv file.  

** TASK 4.1: ** Draw a star schema for this dataset, save your drawing as an image and attach it to the cell below.

Note: We recommended using Google Drawings. Google Drawings is an online image creator that can be used to draw the Schema, and is what the TAs used to create the Schema in the solution.

Tips:
1. Your star schema should be composed of one fact table and three dimension tables. 
1. For each table, underline primary key.
1. Specify data types for each item.
1. Draw 1:N, N:1, N:N or 1:1 on connecting line.
1. Including image in markdown is easy, see [here](https://stackoverflow.com/questions/255170/markdown-and-image-alignment).

### BEGIN SOLUTION
<img src="document/relation.jpg" width = "700" alt="fig_1" align=center />

### END SOLUTION

## Part 4.2: Create database
 
In this part, we will create a [SQLite3 database](https://docs.python.org/3/library/sqlite3.html). This part is really similar to what we saw in lab4b.

The first thing you'll need to do when interacting with a SQLite database (and generally any SQL databases) is to open up a connection (see below). If the database doesn't exist, the SQLite3 module will create one.

In [ ]:
import sqlite3
import csv

conn = sqlite3.connect('movie.db')
c = conn.cursor()

** TASK 4.2: ** Fill in function `create_database` with the following instructions.

- [Drop all existing tables](https://www.techonthenet.com/sqlite/tables/drop_table.php) in the database. Doing this will allow us to simply re-run the script if we want to try a new schema.
- [Create the tables](http://www.sqlitetutorial.net/sqlite-create-table) according to your star schema. 

**Note:** 

- To execute queries, simply run the `cursor.execute()` with your query as the argument.
- Remember to to run 'conn.commit()' in order to commit (i.e. save) your changes.
- Remember to define PRIMARY KEYs and FOREIGN KEYs in your table, refer to [here](http://www.sqlitetutorial.net/sqlite-primary-key/).
- Remember to commit the connection to save changes.

In [ ]:
def create_database(conn):
    """
    Using the database connection conn, create tables appropriate for 
    storing and using IMDB data.  Drops any pre-exsiting tables that would be re-defined by this procedure.
    """
    c = conn.cursor()
    ### BEGIN SOLUTION
    # Drop existing table
    c.execute('DROP TABLE IF EXISTS "movies";')
    c.execute('DROP TABLE IF EXISTS "actors";')
    c.execute('DROP TABLE IF EXISTS "directors";')
    c.execute('DROP TABLE IF EXISTS "facts";')
    # Create database
    c.execute('''
            CREATE TABLE actors(
                actor_id INTEGER,
                actor_name text,
                actor_facebook_likes int,
                PRIMARY KEY(actor_id),
                UNIQUE (actor_name))
                ''')
    c.execute('''
            CREATE TABLE directors(
                director_id INTEGER,
                director_name text,
                director_facebook_likes int,
                PRIMARY KEY(director_id),
                UNIQUE (director_name))''')
    c.execute('''
            CREATE TABLE movies(
                movie_id INTEGER,
                movie_title text,
                num_voted_users int,
                num_user_for_reviews int,
                duration int,
                gross int,
                genre text,
                movie_facebook_likes int,
                cast_total_facebook_likes int,
                facenumber_in_poster int,
                language text,
                country text,
                budget int,
                title_year int,
                imdb_score float,
                aspect_ratio float,
                PRIMARY KEY(movie_id),
                UNIQUE (movie_title))''')
    c.execute('''
            CREATE TABLE facts(
                movie_id INTEGER,
                director_id int not null,
                actor_1_id int not null,
                actor_2_id int not null,
                actor_3_id int not null,
                FOREIGN KEY (movie_id) REFERENCES movies(movie_id),
                FOREIGN KEY (director_id) REFERENCES directors(director_id),
                FOREIGN KEY (actor_1_id) REFERENCES actors(actor_id),
                FOREIGN KEY (actor_2_id) REFERENCES actors(actor_id),
                FOREIGN KEY (actor_3_id) REFERENCES actors(actor_id))''')
    conn.commit()
    ### END SOLUTION
create_database(conn)

Now, you should be able to view the sqlite3 database which you had created in your assignment directory. Play around with the database! 

You can connect to a sqlite3 database via terminal with the following command:

~~~~
sqlite3 move.db
~~~~

To see the schema of your table, you can run the following command:

~~~~
.schema
~~~~

You can execute arbitrary SQL commands from the `sqlite>` prompt.
More info on your other options can be found here.

~~~~
.help
~~~~

Finally, to exit from a sqlite3 database use the following command:

~~~~
.exit
~~~~

## Part 4.3: Populate database

In **Lab 4b**  we provided you with three different csv files, one for each table. However, this isn't how real world data is typically structured when you first receive it. 

Often, you will receive a single data "dump" file. As a data scientist, you need to figure out how to split it into different tables, and insert the information into a database. 

Fill in function `populate_database` to insert the csv file contents into the movie database.  Below is a listing of column definitions that you might find helpful.

~~~~
0 director_name
1 num_critic_for_reviews
2 duration
3 director_facebook_likes
4 actor_3_facebook_likes
5 actor_2_name
6 actor_1_facebook_likes
7 gross
8 genres
9 actor_1_name
10 movie_title
11 num_voted_users
12 cast_total_facebook_likes
13 actor_3_name
14 facenumber_in_poster
15 num_user_for_reviews
16 language
17 country
18 content_rating
19 budget
20 title_year
21 actor_2_facebook_likes
22 imdb_score
23 aspect_ratio
24 movie_facebook_likes
~~~~

In [ ]:
def populate_database(conn, csv_file):    
    reader = csv.reader(open(csv_file, encoding='latin1'))
    
    next(reader, None)
    for row in reader:
        row = [item.lower() for item in row]
        
        # insert into dimension tables
        ### BEGIN SOLUTION
        try:
            c.execute('''
                    INSERT INTO movies (movie_title, num_voted_users, num_user_for_reviews, 
                    duration, gross, genre, movie_facebook_likes, cast_total_facebook_likes, 
                    facenumber_in_poster, language, country, budget, title_year, imdb_score, 
                    aspect_ratio)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                    (row[10], row[11], row[15], row[2], row[7], row[8], 
                    row[24], row[12], row[14], row[16], row[17], row[19], row[20], row[22], 
                    row[23]))
        except sqlite3.IntegrityError:
            pass
        except sqlite3.Error as e:
            print("movies, insert error:", e.args[0])
  
        def insert_likes(table, agent, name, likes):
            try:
                c.execute('''
                        INSERT INTO {0} ({1}_name, {1}_facebook_likes)
                        VALUES (?, ?)'''.format(table, agent), (name, likes))
            except sqlite3.IntegrityError:
                pass
            except sqlite3.Error as e:
                print("%s, insert error:"%table, e.args[0])
                      
        insert_likes('directors', 'director', row[0],row[3])
        insert_likes('actors', 'actor', row[9],row[6])
        insert_likes('actors', 'actor', row[5],row[21])
        insert_likes('actors', 'actor', row[13],row[4])
        ### END SOLUTION
        
        # insert fact table
        ### BEGIN SOLUTION
        c.execute('''
                INSERT INTO facts 
                SELECT m.movie_id, d.director_id, a1.actor_id, a2.actor_id, a3.actor_id
                FROM movies m, directors d, actors a1, actors a2, actors a3
                WHERE m.movie_title = ? AND d.director_name = ? AND a1.actor_name = ? 
                AND a2.actor_name = ? AND a3.actor_name = ?
                ''', (row[10], row[0], row[9], row[5], row[13]))
        ### END SOLUTION
        conn.commit()
        
populate_database(conn, 'all_data.csv')

## Part 4.4: Relational Algebra

In this part, your task is to write relational algebra expressions based on the following three tables that represent a very limited subset of the IMDB dataset. 

`actors` Table:

| name | country | facebook_likes |age|               
|---------------|------|------|------| 
| Jennifer Lawrence| U.S. |34000 |26 | 
|  Johnny Depp  | U.S.| 40000| 54|                    
|Daniel Radcliffe | U.K.|11000 | 28|
| Leonardo DiCaprio|U.S. |29000 | 42| 
|Robin Wright | U.S.|18000 | 51|
|Josh Hutcherson |U.S. |14000 | 24| 
| Peter Dinklage| U.S.|22000 |48 | 

`actor_movies` Table:

| name| movie_name|
|------|------|
| Jennifer Lawrence| X-Men: Days of Future Past |
|Johnny Depp|Pirates of the Caribbean: At World's End|
|Robin Wright | The Girl with the Dragon Tattoo |
| Jennifer Lawrence| The Hunger Games: Mockingjay - Part 2 |
|Leonardo DiCaprio | Inception |
| Peter Dinklage|X-Men: Days of Future Past |
|Josh Hutcherson | The Hunger Games: Mockingjay - Part 2|

`directors` Table:

| name | country | facebook_likes |age|               
|---------------|------|------|------|  
| Christopher Nolan| U.K. |22000 |46 | 
| Alfonso Cuarón   | Mexico| 10000| 55|                    
|Shane Black | U.S.|1000 | 55|
| David Fincher|U.S. |21000 | 54| 
|J.J. Abrams | U.S.|14000 | 51|

`director_movies` Table:

| name| movie_name|
|------|------|
| Christopher Nolan| Inception |
|Shane Black|Iron Man 3|
|Alfonso Cuarón | Gravity |
| J.J. Abrams| Star Wars: The Force Awakens|
|David Fincher | The Girl with the Dragon Tattoo |

#### TASK 4: Based on the tables above, answer the following using relational algebra.
1. Find the name of all movies that have leading american actors who are over 50-years-old.
1. Find the name of all actors/actress that didn't star in the movie _Inception_.
1. Find the name of all movies that have both famous directors and famous actors. (Assume people with at least 20,000 facebook are famous.)
1. Find the name of actor/actress that has the highest number of movies.<br>
    Hint: use aggregate functions
1. Find the name of movies whose actor has the highest number of facebook likes.<br>
    Hint: use aggregate functions/temp variables

_write your answers in this cell_

### BEGIN SOLUTION
1. $\pi_{movie\_name}(\sigma_{country='U.S.' \land age>50}(actors)\bowtie actor\_movies)$
1. $\pi_{name}(actors) - \pi_{name}(\sigma_{movie\_name='Inception'}(actor\_movies))$
1. $\pi_{movie\_name}(actor\_movies \bowtie \sigma_{facebook\_likes>20000}(actors)) \bowtie \pi_{movie\_name}(director\_movies \bowtie \sigma_{facebook\_likes>20000}(directors))$
1. $\pi_{name}({}_{name}g_{max(count)}({}_{name}g_{count(movie\_name) \ AS \ count}(actor\_movies))$
1. $T = \pi_{movie\_name}({}_{name}g_{max(facebook\_likes)}(actors) \bowtie actor\_movies)$

### END SOLUTION

***
## Part 4.5: Running Queries

Now that you have created a database with the IMDB data. Your customer needs records that satisfy their requirements based on several constraints in order to improve their business revenue. 
In this part, you will create several SQL statements to extract them. 

IPython has a built-in module called [sql](https://pypi.python.org/pypi/ipython-sql) which allows us to easily interface with a database. The following two lines load this SQL extension and connect to your IMDB database:

In [ ]:
!pip install ipython-sql

In [ ]:
%reload_ext sql
%sql sqlite:///movie.db

Your task is to write 10 queries that fetch certain records for your customers. To run SQL queries, first call <font color = "red">%%sql</font> magic and then write your SQL queries.

**Query 4.1: Find the number of directors in the directors table.**


In [ ]:
%%sql
SELECT count(director_name) AS count FROM directors


**Query 4.2: Find the number of actors in the actors table.**


In [ ]:
%%sql
SELECT count(actor_name) AS count FROM actors


**Query 4.3: Find the number of movies in the movies table.**


In [ ]:
%%sql
SELECT count(*) AS count FROM movies


There should be 1453 directors, 3872 actors, 3023 movies. Please make sure you get the same results before running the following queries.

Now, we can have some fun querying the database!

**Query 4.4: Find years that have more than 100 movies and arrange them in ascending order. **


In [ ]:
%%sql
SELECT title_year
FROM movies
GROUP BY title_year
HAVING COUNT(*) >= 100
ORDER BY title_year;


**Query 4.5: Order director name from most Facebook likes to fewest Facebook likes and only show top 10 directors.**


In [ ]:
%%sql
SELECT director_name
FROM directors
ORDER BY director_facebook_likes DESC
LIMIT 10;


**Query 6: Find the titles of movies that are directed by Christopher Nolan and order them by movie title. **


In [ ]:
%%sql
SELECT movie_title
FROM directors d, movies m, facts f
WHERE f.movie_id=m.movie_id AND 
f.director_id=d.director_id AND 
d.director_name='christopher nolan'
ORDER BY movie_title;


** Query 4.7: Find all the directors who directed at least 5 movies since 2007 (included) sorted by the number of movies in descending order.**


In [ ]:
%%sql
SELECT tmp.director_name, tmp.count
FROM (SELECT d.director_id, d.director_name, COUNT(*) AS count
      FROM directors d, movies m, facts f
      WHERE f.movie_id=m.movie_id AND f.director_id=d.director_id AND m.title_year>=2007
      GROUP BY d.director_id, d.director_name) AS tmp
WHERE tmp.count>=5
ORDER BY tmp.count desc;


**Query 4.8: Find the top five most productive actors since 2010 (included).**


In [ ]:
%%sql
SELECT tmp.actor_name, tmp.count
FROM (SELECT a.actor_id, a.actor_name, COUNT(*) AS count
      FROM actors a, movies m, facts f
      WHERE f.movie_id=m.
      movie_id AND m.title_year>=2010 AND (f.actor_1_id=a.actor_id OR f.actor_2_id=a.actor_id OR f.actor_3_id=a.actor_id)
      GROUP BY a.actor_id, a.actor_name) AS tmp
ORDER BY tmp.count desc
LIMIT 5;


**Query 4.9: Find all the actors who have collaborated with Christopher Nolan, sorted by their collaboration frequency in descending order.**


In [ ]:
%%sql
SELECT tmp.actor_name, COUNT(movie_id) AS count
FROM (SELECT actor_1_id AS actor_id, actor_name, movie_id FROM directors d, facts f, actors a
      WHERE d.director_id=f.director_id AND d.director_name='christopher nolan' AND f.actor_1_id=a.actor_id
      UNION
      SELECT actor_2_id AS actor_id, actor_name, movie_id FROM directors d, facts f, actors a
      WHERE d.director_id=f.director_id AND d.director_name='christopher nolan' AND f.actor_2_id=a.actor_id
      UNION
      SELECT actor_3_id AS actor_id, actor_name, movie_id FROM directors d, facts f, actors a
      WHERE d.director_id=f.director_id AND d.director_name='christopher nolan' AND f.actor_3_id=a.actor_id) AS tmp
GROUP BY tmp.actor_name
ORDER BY count DESC;


**Query 4.10: Without using an aggregation operator, write a SQL statement that finds the title of the movie(s) with the most Facebook likes.**


In [ ]:
%%sql
SELECT m.movie_title
FROM movies m
EXCEPT
SELECT m1.movie_title 
FROM movies m1, movies m2
WHERE m1.movie_facebook_likes < m2.movie_facebook_likes;

### END SOLUTION

## Part 4.6: Database Visualization

Since your customer isn't very good with databases, SQL or tabluar data,  they want your help visualizing some of database. With the help of SQL, you can easily retrieve data from the database, and then use pandas dataframes and `matlabplotlib.pyplot` to create the visualizations.

Note: refer to Pandas visualization method [here](https://pandas.pydata.org/pandas-docs/stable/visualization.html).

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

** Example **:
The customer wants a a bar chart of the top 10 actors(based on facebook likes), sorted in descending order. 

In [ ]:
# Solution

c.execute('''
        SELECT actor_name, actor_facebook_likes
        FROM actors
        ORDER BY actor_facebook_likes DESC
        LIMIT 10''')
dataframe = pd.DataFrame(data = c.fetchall(), columns = ['actor_name', 'actor_fcebook_likes'])
dataframe.plot.bar(x='actor_name', title='actor facebook likes', figsize=(10,7))

** Task 4.5 **: Create box plots of movie duration, imdb_score and num_critic_for_reviews.  Hint: Create each in its own plot, otherwise the y-axis will not be informative.

In [ ]:
### BEGIN SOLUTION
c = conn.cursor()
c.execute('''
        SELECT duration, imdb_score, num_user_for_reviews
        FROM movies''')
cols = ['duration', 'imdb_score', 'num_user_for_reviews'];
df = pd.DataFrame(data = c.fetchall(), columns = cols)
df.describe()
plt.subplot(311)
df['duration'].plot.box(figsize=(10,7), sym='+')
plt.subplot(312)
df['imdb_score'].plot.box(figsize=(10,7), sym='+')
plt.subplot(313)
df['num_user_for_reviews'].plot.box(figsize=(10,7), sym='+')
### END SOLUTION

** Task 4.6 **: Create a pie chart of movie counts per country of countries that have more than 10 movies

In [ ]:
### BEGIN SOLUTION
c.execute('''
        SELECT country, COUNT(country) AS number
        FROM movies
        GROUP BY country
        HAVING COUNT(country) > 10''')
dataframe = pd.DataFrame(data = c.fetchall(), columns = ['country', 'number'])
dataframe.plot.pie(figsize=(10,10), y = 'number', labels = dataframe['country'].tolist())
### END SOLUTION

** Task 4.7 **: Plot a curve of cumulative number of movies released over time.

** hint **: use [cumsum](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.cumsum.html) function.

In [ ]:
### BEGIN SOLUTION
c.execute('''
        SELECT title_year, COUNT(*) AS number
        FROM movies
        GROUP BY title_year
        ORDER BY title_year''')
dataframe = pd.DataFrame(data = c.fetchall(), columns = ['title_year', 'number'])
dataframe['sum'] = dataframe['number'].cumsum()
dataframe.plot(x='title_year', y='sum', title = 'Total movies', figsize=(9, 7))
### END SOLUTION

***
## Part 4.7: Query Optimization
In this part, we will see how to optimize database access by indexing and using materialized views. 

The next three functions create and exercise a toy database.
`setup_test` creates a connection to the database.  `insert_test` performs `num` SQL `SELECT`s, the  function `select_test` does `num` SQL `INSERT`s.  

In [ ]:
def setup_test(db):
    conn = sqlite3.connect(db)
    conn.execute('DROP TABLE IF EXISTS scripts')
    conn.execute('''
            CREATE TABLE scripts (
                script_id INTEGER,
                author_name text,
                script_name text,
                author_facebook_likes int,
                PRIMARY KEY(script_id))
                ''')
    conn.commit()
    return conn
                

def insert_test(conn, num):
    c = conn.cursor()
    for i in range(num):
        c.execute('''INSERT INTO scripts (script_name) VALUES ('Dr. Who %d')'''%i)
    conn.commit()

def select_test(conn, num):
    c = conn.cursor()
    for i in range(num):
        c.execute('''SELECT * 
                     FROM scripts
                     WHERE script_name = "Dr. Who %d"'''%i)



Below is some code for benchmarking your database.  Run it to see how well your database currently performs.

In [ ]:
conn = setup_test('test.db')
insert_test(conn, 15000)

In [ ]:
%timeit -n 3 -r 3 insert_test(conn, 15000)

In [ ]:
%timeit -n 3 -r 3 select_test(conn, 5000) 

### Indexing

** Task 4.8 **: Optimize the database so that `test_select` test performs more quickly.  Fill in the appropriate SQL commands in the cell below.  

Hint: try adding an INDEX

Note: The line `!cp movie.db movie1.db` copies your `movie.db` database to `movie1.db`.

In [ ]:
conn1 = setup_test('test1.db')

### BEGIN SOLUTION
conn1.execute('''CREATE INDEX IF NOT EXISTS script_name_index ON scripts(script_name)''')
### END SOLUTION
conn1.commit()

Run the following code to see how your optimization performs.

In [ ]:
%timeit -n 3 -r 3 insert_test(conn1,15000)

In [ ]:
%timeit -n 3 -r 3 select_test(conn1, 5000) 

** Task 4.9 **: Explain the performance changes you see.  Have any trade-offs been made?

### BEGIN SOLUTION
The read performance is much faster (more than an order of magnitude quicker).  The write performance is slower by about a factor of two.  This occurs because the index allows searching to occur much more quickly (think binary search or hash indexing) whereas maintaining the index costs more in time.
### END SOLUTION

### Temporary Tables
Looking at the source code you have developed for the company you notice the following query turns out to be a bottle neck, 

```
SELECT * FROM actors a, directors d, movies m, facts f 
                WHERE a.actor_id = f.actor_1_id AND d.director_id = f.director_id AND m.movie_id = f.movie_id
```

Below is some code to benchmark it's performance.  Try running it.

In [ ]:
def test(conn, num):
    c = conn.cursor()
    for i in range(num):
        c.execute(
            '''SELECT * FROM actors a, directors d, movies m, facts f 
                WHERE a.actor_id = f.actor_1_id AND d.director_id = f.director_id AND m.movie_id = f.movie_id''')
        conn.commit()

In [ ]:
conn = sqlite3.connect('movie.db')
%timeit test(conn,50000)

** Task 4.10: ** One way to speed this query up is to store it's results and refer to them as needed, as opposed to rerunning the query.  Do this *inside* the database by using SQL to create a tempory table called 'testtmp' of these search results.  Write solution below, and then use the benchmarking code provided to test it.

In [ ]:
def setup_mytmp(conn):
    ''' Create a temporary table for a common query's results'''
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS mytmp''')
### BEGIN SOLUTION
    c.execute('''CREATE TABLE IF NOT EXISTS mytmp AS 
             SELECT * FROM actors a, directors d, movies m, facts f 
            WHERE a.actor_id = f.actor_1_id AND d.director_id = f.director_id AND m.movie_id = f.movie_id''')
    conn.commit()
### END SOLUTION

In [ ]:
def test_mytmp(conn, num):
    c = conn.cursor()
    setup_mytmp(conn)
    for i in range(num):
        c.execute('''SELECT * FROM mytmp''')

        
%timeit test_mytmp(conn, 50000)

### Analysis

In this example, the run time of each test depends on the parameter `num`. The code below runs both `test` and your optimized version `test_mytmp` with `num = [100, 200, 400, 800, 1600, 3200, 6400]` and plots the run time performance of each function.  Note: This function uses `timeit` to find best execution times using repeated trials, the number of trails used depends on each trails run time (shorter times use more trials).

In [ ]:
import timeit as timeit

X = [100, 200, 400, 800, 1600, 3200, 6400]
a = []
b = []
for i in X:
    result = %timeit -o test(conn,i)
    a.append(result.best)
    result = %timeit -o test_mytmp(conn,i)
    b.append(result.best)     
    
plt.plot(X, a, label = 'test')
plt.plot(X, b, label = 'test_mytmp')
plt.xlabel = 'number of select'
plt.ylabel = 'time / s'
plt.legend()

** Task 4.11 ** What do the performance curves tell you about the use of temporary tables in this example? (you may need to run this code a few times to get a good feel for the situation)

_write your answer here_